In [ ]:
%matplotlib inline


Speech Recognition with Wav2Vec2
================================

**Author**: `Moto Hira <moto@fb.com>`__

**번역자** : `이종법 <bub3690@naver.com>`__



번역문 :

이 튜토리얼은 미리 학습된(pre-trained) wav2vec 2.0 모델을 사용하여 어떻게 음성 인식을 수행하는지 보여줍니다.

---


원문:

This tutorial shows how to perform speech recognition using using
pre-trained models from wav2vec 2.0
[`paper <https://arxiv.org/abs/2006.11477>`__].


Overview
--------



번역문 :

음성 인식의 과정은 다음과 유사합니다.

1. 오디오 파형으로부터 음향 특징을 추출한다.

2. 각 프레임별로 음향 특징의 범주를 예측한다. 

3. 연속된 범주 확률들로 부터 가설을 생성한다.


Torchaudio는 미리 학습된 가중치들과 예상되는 샘플링 비율, 범주 레이블과 같은 관련 정보에 쉽게 접근하게 해줍니다.
그것들은 함께 묶여, :py:func:`torchaudio.pipelines` 모듈에서 사용가능합니다.

---

원문 :


The process of speech recognition looks like the following.

1. Extract the acoustic features from audio waveform

2. Estimate the class of the acoustic features frame-by-frame

3. Generate hypothesis from the sequence of the class probabilities

Torchaudio provides easy access to the pre-trained weights and
associated information, such as the expected sample rate and class
labels. They are bundled together and available under
:py:func:`torchaudio.pipelines` module.




Preparation
-----------
번역문:

가장 먼저 필수 패키지를  불러오고, 우리가 작업할 데이터를 받아옵니다.

---

원문 : 

First we import the necessary packages, and fetch data that we work on.




In [ ]:
# %matplotlib inline

import os

import IPython
import matplotlib
import matplotlib.pyplot as plt
import requests
import torch
import torchaudio

matplotlib.rcParams["figure.figsize"] = [16.0, 4.8]

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.__version__)
print(torchaudio.__version__)
print(device)

SPEECH_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"  # noqa: E501
SPEECH_FILE = "_assets/speech.wav"

if not os.path.exists(SPEECH_FILE):
    os.makedirs("_assets", exist_ok=True)
    with open(SPEECH_FILE, "wb") as file:
        file.write(requests.get(SPEECH_URL).content)

Creating a pipeline
-------------------

번역문 :

가장 먼저, 특징 추출과 분류를 수행하는 Wav2Vec2 모델을 생성합니다.

torchaudio 에서 미리 학습된 Wav2Vec2 가중치들은 두가지가 있습니다. 하나는 자동 음성 인식(ASR)에 미리 학습된 것, 다른 하나는 미리 학습되지 않은 것입니다.

Wav2Vec2 ( 그리고 HuBERT) 모델들은 자기-지도학습 방법으로 학습됩니다. 그 모델들은 가장 먼저 표현 학습만을 위해 오디오를 사용하여 학습되고, 그리고 나서 추가적인 레이블로 특정 업무를 미세조정합니다.

또한 미세 조정없이 미리 학습된 가중치들은 다른 적용하려는 업무(downstream task)를 미세조정 할 수 있습니다, 그러나 이 튜토리얼은 그것을 다루지 않습니다.

우리는 이 여기서 :py:func:`torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H` 을 사용할 것입니다.

:py:mod:`torchaudio.pipelines` 과 같은 많은 모델들이 제공됩니다. 어떻게 학습되는지 자세한 방법은 문서를 참조해주세요.

묶음(bundle) 객체들은 모델을 객체를 생성하는 인터페이스와 다른 정보들을 제공합니다. 샘플링 비율과 범주 레이블은 다음과 같이 볼 수 있습니다. 



---

원문 : 

First, we will create a Wav2Vec2 model that performs the feature
extraction and the classification.

There are two types of Wav2Vec2 pre-trained weights available in
torchaudio. The ones fine-tuned for ASR task, and the ones not
fine-tuned.

Wav2Vec2 (and HuBERT) models are trained in self-supervised manner. They
are firstly trained with audio only for representation learning, then
fine-tuned for a specific task with additional labels.

The pre-trained weights without fine-tuning can be fine-tuned
for other downstream tasks as well, but this tutorial does not
cover that.

We will use :py:func:`torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H` here.

There are multiple models available as
:py:mod:`torchaudio.pipelines`. Please check the documentation for
the detail of how they are trained.

The bundle object provides the interface to instantiate model and other
information. Sampling rate and the class labels are found as follow.




In [ ]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H

print("Sample Rate:", bundle.sample_rate)

print("Labels:", bundle.get_labels())

번역문 :

모델은 다음과 같이 구축될 수 있습니다. 이 과정은 자동으로 미리 학습된 가중치를 불러오고, 모델에 탑재 해줍니다.


---

원문 :

Model can be constructed as following. This process will automatically
fetch the pre-trained weights and load it into the model.




In [ ]:
model = bundle.get_model().to(device)

print(model.__class__)

Loading data
------------

번역문 :

우리는 `VOiCES dataset <https://iqtlabs.github.io/voices/>`__,의 음성 데이터를 사용할 것 입니다.


---

원문 :

We will use the speech data from `VOiCES
dataset <https://iqtlabs.github.io/voices/>`__, which is licensed under
Creative Commos BY 4.0.




In [ ]:
IPython.display.Audio(SPEECH_FILE)

번역문 : 

데이터를 탑재 하기위해서는, :py:func:`torchaudio.load`을 사용합니다.

만약 샘플링 비율이 파이프라인(pipeline)이 예상하는 것과 다르다면, :py:func:`torchaudio.functional.resample` 을 사용하여 다시 샘플링 할 수 있습니다.

<div class="alert alert-info"><h4>추가 정보</h4><p>- :py:func:`torchaudio.functional.resample` CUDA 텐서에서도 사용 가능합니다.
   - 같은 샘플링 비율들의 집합을 여러번 다시 샘플링 하는 경우에는,
     :py:func:`torchaudio.transforms.Resample`을 사용하는 것이 성능을 더 높여줄 수 있습니다.</p></div>


---

원문 : 


To load data, we use :py:func:`torchaudio.load`.

If the sampling rate is different from what the pipeline expects, then
we can use :py:func:`torchaudio.functional.resample` for resampling.

<div class="alert alert-info"><h4>Note</h4><p>- :py:func:`torchaudio.functional.resample` works on CUDA tensors as well.
   - When performing resampling multiple times on the same set of sample rates,
     using :py:func:`torchaudio.transforms.Resample` might improve the performace.</p></div>




In [ ]:
waveform, sample_rate = torchaudio.load(SPEECH_FILE)
waveform = waveform.to(device)

if sample_rate != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

Extracting acoustic features
----------------------------

번역문 : 

다음 단계는 오디오로 부터 음향 특징을 추출하는 것입니다.

<div class="alert alert-info"><h4>추가 정보</h4><p>자동-음성-인식(ASR) 업무에 미세조정된 Wav2Vec2 모델은 특징 추출과 분류를 한번에 수행가능하지만, 튜토리얼을 위해, 여기서는 어떻게 특징 추출하는지 보여드립니다.</p></div>

---

원문 :

The next step is to extract acoustic features from the audio.

<div class="alert alert-info"><h4>Note</h4><p>Wav2Vec2 models fine-tuned for ASR task can perform feature
   extraction and classification with one step, but for the sake of the
   tutorial, we also show how to perform feature extraction here.</p></div>




In [ ]:
with torch.inference_mode():
    features, _ = model.extract_features(waveform)

번역문 :

특징들은 텐서의 리스트로 반환됩니다. 각 텐서는 트랜스폼 계층(Transform layer)의 출력입니다. 

---

원문 : 

The returned features is a list of tensors. Each tensor is the output of
a transformer layer.




In [ ]:
fig, ax = plt.subplots(len(features), 1, figsize=(16, 4.3 * len(features)))
for i, feats in enumerate(features):
    ax[i].imshow(feats[0].cpu())
    ax[i].set_title(f"Feature from transformer layer {i+1}")
    ax[i].set_xlabel("Feature dimension")
    ax[i].set_ylabel("Frame (time-axis)")
plt.tight_layout()
plt.show()

Feature classification
----------------------

번역문 : 

음향 특징들이 추출되면, 다음 단계는 특징들을 범주들의 집합으로 분류하는 것입니다.

Wav2Vec2 모델은 특징 추출과 분류를 한번에 수행하는 방법을 제공합니다.


---

원문 :

Once the acoustic features are extracted, the next step is to classify
them into a set of categories.

Wav2Vec2 model provides method to perform the feature extraction and
classification in one step.




In [ ]:
with torch.inference_mode():
    emission, _ = model(waveform)

번역문 : 

출력값은 로짓의 형태입니다. 이것은 확률의 형태가 아닙니다.

이것을 시각화 해봅시다.

---

원문 : 

The output is in the form of logits. It is not in the form of
probability.

Let’s visualize this.




In [ ]:
plt.imshow(emission[0].cpu().T)
plt.title("Classification result")
plt.xlabel("Frame (time-axis)")
plt.ylabel("Class")
plt.show()
print("Class labels:", bundle.get_labels())

번역문 : 

시간 축을 따라 특정 레이블에 강한 표시들을 볼 수 있습니다.


---

원문 : 

We can see that there are strong indications to certain labels across
the time line.




Generating transcripts
----------------------
번역문 : 

연속된 레이블 확률들로 부터, 이제 우리는 글을 생성하고 싶습니다. 가설을 생성하는 과정은 종종 "복호화(decoding)" 이라고도 불립니다.


복호화는 단순 분류보다 더 정교한데, 왜냐하면 특정 시간 단계에서 복호화 하는 것은 주변 관측 값들에 영향을 받기 때문입니다.

``night``와 ``knight``를 예로 들자. 비록 그들의 사전 확률분포는 다르지만 (전형적인 대화에서 ``night``가 더 많이 등장한다.), ``a knight with a sword``와 같이 ``knight``가 있는 글을 정확히 생성하기 위해서, 복호화 작업은 충분한 문맥을 보기전까지  최종 결정을 연기해야한다.  

많은 복호화 기술들이 제안되었고, 그 기술들은 단어 사전이나 언어 모델(language model) 처럼 외부 자료를 필요로 합니다.

이 튜토리얼에서는, 단순화를 위해, 탐욕(Greedy) 복호화를 수행한다. 탐욕 보호화는 그런 외부 요소들에 의존하지않고, 간단히 매 시간 단계마다 가장 최선의 가설을 선택한다. 그러므로, 문맥 정보는 사용되지 않고, 오직 한 글만 생성이 될 수 있다.

우리는 탐욕 복호화 알고리즘 정의부터 시작하겠습니다.


---

원문 : 


From the sequence of label probabilities, now we want to generate
transcripts. The process to generate hypotheses is often called
“decoding”.

Decoding is more elaborate than simple classification because
decoding at certain time step can be affected by surrounding
observations.

For example, take a word like ``night`` and ``knight``. Even if their
prior probability distribution are differnt (in typical conversations,
``night`` would occur way more often than ``knight``), to accurately
generate transcripts with ``knight``, such as ``a knight with a sword``,
the decoding process has to postpone the final decision until it sees
enough context.

There are many decoding techniques proposed, and they require external
resources, such as word dictionary and language models.

In this tutorial, for the sake of simplicity, we will perform greedy
decoding which does not depend on such external components, and simply
pick up the best hypothesis at each time step. Therefore, the context
information are not used, and only one transcript can be generated.

We start by defining greedy decoding algorithm.




In [ ]:
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        super().__init__()
        self.labels = labels
        self.blank = blank

    def forward(self, emission: torch.Tensor) -> str:
        """Given a sequence emission over labels, get the best path string
        Args:
          emission (Tensor): Logit tensors. Shape `[num_seq, num_label]`.

        Returns:
          str: The resulting transcript
        """
        indices = torch.argmax(emission, dim=-1)  # [num_seq,]
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = [i for i in indices if i != self.blank]
        return "".join([self.labels[i] for i in indices])

번역문 : 

복호기 객체를 생성하고 글을 복호화 합니다.


---

원문 : 


Now create the decoder object and decode the transcript.




In [ ]:
decoder = GreedyCTCDecoder(labels=bundle.get_labels())
transcript = decoder(emission[0])

번역문 : 

결과를 확인하고 오디오를 다시 들어보세요.

---
원문 :


Let’s check the result and listen again to the audio.




In [ ]:
print(transcript)
IPython.display.Audio(SPEECH_FILE)

번역문 : 

자동-음성-인식(ASR) 모델은 연결주의-시간적-분류(CTC) 손실 함수에 의해 미세조정된다.

CTC 손실 함수의 자세한 내용은 `here <https://distill.pub/2017/ctc/>`__에서 설명된다.

CTC에서 빈 토큰 (ϵ)은 이전 기호의 반복을 나타내는 특별한 토큰이다. 복호화에서는, 간단히 무시된다. 

---
원문 :

The ASR model is fine-tuned using a loss function called Connectionist Temporal Classification (CTC).
The detail of CTC loss is explained
`here <https://distill.pub/2017/ctc/>`__. In CTC a blank token (ϵ) is a
special token which represents a repetition of the previous symbol. In
decoding, these are simply ignored.




Conclusion
----------

번역문 : 

이 튜토리얼에서는, 우리는 음향 특징 추출, 음성 인식을 위해서 :py:mod:`torchaudio.pipelines`을 어떻게 사용하는지 살펴보았다. 모델을 생성하고 출력을 받는것은 짧게 두줄이다.



---
원문 :

In this tutorial, we looked at how to use :py:mod:`torchaudio.pipelines` to
perform acoustic feature extraction and speech recognition. Constructing
a model and getting the emission is as short as two lines.

::

   model = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H.get_model()
   emission = model(waveforms, ...)


